In this script I try some of the possible ways of correlating the segmentation with the nodes communities

- By using the communities COM and the segmentation
- By using the correlation between the parcellation and the community of provenience


In [1]:
import nibabel as nib
import numpy as np
import networkx as nx
from skimage.measure import marching_cubes
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm
import networkx as nx
import os
import numpy as np
from glob import glob
import nibabel as nib
import nibabel.freesurfer.mghformat as mghf # read mgz images
from skimage.morphology import skeletonize_3d
from skimage.measure import marching_cubes
import matplotlib.pyplot as plt
import pandas as pd



In [2]:

def rescale_coordinates(node_positions, skeleton_coords):
    graph_min = node_positions.min(axis=0)
    graph_max = node_positions.max(axis=0)
    skeleton_min = skeleton_coords.min(axis=0)
    skeleton_max = skeleton_coords.max(axis=0)

    normalized_node_positions = (
        (node_positions - graph_min) / (graph_max - graph_min)
    ) * (skeleton_max - skeleton_min) + skeleton_min

    return normalized_node_positions

In [ ]:
# Path to the main directory containing the folders with data
main_directory = "PR-graphomics/sequences/"

# List all subfolders inside the main directory
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
folder = folders[10]

print(f"Accessing folder: {folder}")
folder_path = os.path.join(main_directory, folder)
    
#In this way I add to the address the folder inside the first one
folder_path = os.path.join(folder_path, os.listdir(folder_path)[0])
#aparc.DKTatlas+aseg.deep.mgz aparc.DKTatlas+aseg.deep.nii.gz
image_path = os.path.join(folder_path, 'aparc.DKTatlas+aseg.deep.mgz')
vol = mghf.load(image_path)


df = pd.read_csv('PR-graphomics/labelmap.csv', sep = ',')
df[0:10]